# What is GDAL ?

In [2]:
from __future__ import print_function

# Import the "gdal" submodule from within the "osgeo" module
from osgeo import gdal

# We can check which version we're running by printing the "__version__" variable
print("GDAL's version is: " + gdal.__version__)
print(gdal)
# Works
print(gdal.GDT_Byte)

GDAL's version is: 3.4.3
<module 'osgeo.gdal' from 'C:\\Users\\furka\\myenvBs\\Lib\\site-packages\\osgeo\\gdal.py'>
1


## Get Info from Image Data

In [4]:
data = gdal.Open('./example/LE70220491999322EDC01_stack.gtif', gdal.GA_ReadOnly)
print(data)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001954CBAC930> >


In [12]:
num_bands = data.RasterCount
print('Number of bands in image: {n}\n'.format(n=num_bands))

rows = data.RasterYSize
cols = data.RasterXSize
print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

# Does the raster have a description or metadata?
desc = data.GetDescription()
metadata = data.GetMetadata()

print('Raster description: {desc}'.format(desc=desc))
print('Raster metadata:')
print(metadata)
print('\n')

# What driver was used to open the raster?
driver = data.GetDriver()
print('Raster driver: {d}\n'.format(d=driver.ShortName))

# What is the raster's projection?
proj = data.GetProjection()
print('Image projection:')
print(proj + '\n')

# What is the raster's "geo-transform"
gt = data.GetGeoTransform()
print('Image geo-transform: {gt}\n'.format(gt=gt))

Number of bands in image: 8

Image size is: 250 rows x 250 columns

Raster description: ./example/LE70220491999322EDC01_stack.gtif
Raster metadata:
{'AREA_OR_POINT': 'Area', 'Band_1': 'band 1 reflectance', 'Band_2': 'band 2 reflectance', 'Band_3': 'band 3 reflectance', 'Band_4': 'band 4 reflectance', 'Band_5': 'band 5 reflectance', 'Band_6': 'band 7 reflectance', 'Band_7': 'band 6 temperature', 'Band_8': 'Band 8'}


Raster driver: GTiff

Image projection:
PROJCS["WGS 84 / UTM zone 15N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Eastin

In [13]:
# Open the blue band in our image
blue = data.GetRasterBand(1)

print(blue)

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000001954D3EEB80> >


In [18]:
# What is the band's datatype?
datatype = blue.DataType
print('Band datatype: {dt}'.format(dt=blue.DataType))

# If you recall from our discussion of enumerated types, this "3" we printed has a more useful definition for us to use
datatype_name = gdal.GetDataTypeName(blue.DataType)
print('Band datatype: {dt}'.format(dt=datatype_name))

# We can also ask how much space does this datatype take up
bytes = gdal.GetDataTypeSize(blue.DataType)
print('Band datatype size: {b} bytes\n'.format(b=bytes))

# How about some band statistics?
band_max, band_min, band_mean, band_stddev = blue.GetStatistics(0, 1)
print('Band range: {minimum} - {maximum}'.format(maximum=band_max, minimum=band_min))
print('Band mean, stddev: {m}, {s}\n'.format(m=band_mean, s=band_stddev))

Band datatype: 3
Band datatype: Int16
Band datatype size: 16 bytes

Band range: 1810.0 - 198.0
Band mean, stddev: 439.015984, 139.7168287663



## Using Numpy

In [19]:
# No alias
import numpy
print(numpy.__version__)

# Alias or rename to "np" -- a very common practice
import numpy as np
print(np.__version__)

1.24.2
1.24.2


### With GDAL

In [20]:
help(blue.ReadAsArray)

Help on method ReadAsArray in module osgeo.gdal:

ReadAsArray(xoff=0, yoff=0, win_xsize=None, win_ysize=None, buf_xsize=None, buf_ysize=None, buf_type=None, buf_obj=None, resample_alg=0, callback=None, callback_data=None) method of osgeo.gdal.Band instance
    Reading a chunk of a GDAL band into a numpy array. The optional (buf_xsize,buf_ysize,buf_type)
    parameters should generally not be specified if buf_obj is specified. The array is returned



In [23]:
blue_data = blue.ReadAsArray()

print(blue_data)
print()
print('Blue band mean is: {m}'.format(m = blue_data.mean()))
print('Size is: {sz}'.format(sz = blue_data.shape))

[[569 526 569 ... 311 289 311]
 [568 589 568 ... 267 332 332]
 [546 525 589 ... 311 311 311]
 ...
 [499 543 478 ... 306 349 372]
 [520 520 543 ... 328 372 393]
 [543 564 543 ... 393 414 436]]

Blue band mean is: 439.015984
Size is: (250, 250)


### With Numpy

In [24]:
# Initialize a 3d array -- use the size properties of our image for portability!
image = np.zeros((data.RasterYSize, data.RasterXSize, data.RasterCount))


In [33]:
# Loop over all bands in dataset
for b in range(data.RasterCount):
    # GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    image[:, :, b] = data.GetRasterBand(b + 1).ReadAsArray()
    
#print(image)
print(image.dtype)


float64


### Another Option To Convert From Gdal to Numpy 

In [34]:
data.GetRasterBand(1).DataType

3

In [35]:
from osgeo import gdal_array

# DataType is a property of the individual raster bands
image_datatype = data.GetRasterBand(1).DataType

# Allocate our array, but in a more efficient way
image_correct = np.zeros(
    (data.RasterYSize, data.RasterXSize, data.RasterCount),
    dtype = gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype)
) 

# Loop over all bands in dataset
for b in range(data.RasterCount):
    # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    band = data.GetRasterBand(b + 1)
    
    # Read in the band's data into the third dimension of our array
    image_correct[:, :, b] = band.ReadAsArray()

print("Compare datatypes: ")
print("    when unspecified: {dt}".format(dt=image.dtype))
print("    when specified: {dt}".format(dt=image_correct.dtype))

Compare datatypes: 
    when unspecified: float64
    when specified: int16


In [36]:
dataset = None

image = None
image_correct = None